In [1]:
# Build model with non-linear activation function
from torch import nn
class CircleModelV2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=11, out_features=40)
        self.layer_2 = nn.Linear(in_features=40, out_features=40)
        self.layer_3 = nn.Linear(in_features=40, out_features=1)
        self.relu = nn.ReLU() # <- add in ReLU activation function
        # Can also put sigmoid in the model 
        # This would mean you don't need to use it on the predictions
        # self.sigmoid = nn.Sigmoid()

    def forward(self, x):
      # Intersperse the ReLU activation function between layers
       return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))

model_3 = CircleModelV2().to('cuda')
print(model_3)

CircleModelV2(
  (layer_1): Linear(in_features=11, out_features=40, bias=True)
  (layer_2): Linear(in_features=40, out_features=40, bias=True)
  (layer_3): Linear(in_features=40, out_features=1, bias=True)
  (relu): ReLU()
)


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
dataset = pd.read_csv('loan_approval_dataset.csv',index_col=0)

In [3]:
dataset.head()

,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
loan_id,,,,,,,,,,,,
1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected


In [4]:
from sklearn import preprocessing


label_encoder = preprocessing.LabelEncoder()
cols = [" education", " self_employed", " loan_status"]
for col in cols:
    dataset[col] = label_encoder.fit_transform(dataset[col])
X = dataset.drop([' loan_status'], axis=1)
y = dataset[' loan_status']

In [5]:
X,y

(         no_of_dependents  education  self_employed  income_annum  \
 loan_id                                                             
 1                       2          0              0       9600000   
 2                       0          1              1       4100000   
 3                       3          0              0       9100000   
 4                       3          0              0       8200000   
 5                       5          1              1       9800000   
 ...                   ...        ...            ...           ...   
 4265                    5          0              1       1000000   
 4266                    0          1              1       3300000   
 4267                    2          1              0       6500000   
 4268                    1          1              0       4100000   
 4269                    1          0              0       9200000   
 
          loan_amount  loan_term  cibil_score  residential_assets_value  \
 loan_id     

In [6]:
num_cols = X.select_dtypes(include=np.number).columns.tolist()

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(dataset[num_cols])

In [ ]:
X[num_cols] = scaler.transform(X[num_cols])

In [9]:
import torch
X_tensor = torch.tensor(X.values).type(torch.float)
y_tensor = torch.tensor(y.values).type(torch.float)

In [10]:
X_tensor[:5], y_tensor[:5]

(tensor([[0.4000, 0.0000, 0.0000, 0.9691, 0.7551, 0.5556, 0.7967, 0.0856, 0.9072,
          0.5758, 0.5442],
         [0.0000, 1.0000, 1.0000, 0.4021, 0.3036, 0.3333, 0.1950, 0.0959, 0.1134,
          0.2185, 0.2245],
         [0.6000, 0.0000, 0.0000, 0.9175, 0.7500, 1.0000, 0.3433, 0.2466, 0.2320,
          0.8483, 0.8707],
         [0.6000, 0.0000, 0.0000, 0.8247, 0.7755, 0.3333, 0.2783, 0.6267, 0.1701,
          0.5913, 0.5374],
         [1.0000, 1.0000, 1.0000, 0.9897, 0.6097, 1.0000, 0.1367, 0.4281, 0.4227,
          0.7481, 0.3401]]),
 tensor([0., 1., 1., 1., 1.]))

In [11]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tensor, 
                                                    y_tensor, 
                                                    test_size=0.2, # 20% test, 80% train
                                                    random_state=42) # make the random split reproducible

len(X_train), len(X_test), len(y_train), len(y_test)

(3415, 854, 3415, 854)

In [12]:
# Setup loss and optimizer 
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model_3.parameters(), lr=0.1)

In [13]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [14]:
# Fit the model
torch.manual_seed(42)
epochs = 1000

# Put all data on target device
X_train, y_train = X_train.to("cuda"), y_train.to("cuda")
X_test, y_test = X_test.to("cuda"), y_test.to("cuda")

for epoch in range(epochs):
    # 1. Forward pass
    y_logits = model_3(X_train).squeeze()
    y_pred = torch.round(torch.sigmoid(y_logits)) # logits -> prediction probabilities -> prediction labels
    
    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train) # BCEWithLogitsLoss calculates loss using logits
    acc = accuracy_fn(y_true=y_train, 
                      y_pred=y_pred)
    
    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_3.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_logits = model_3(X_test).squeeze()
      test_pred = torch.round(torch.sigmoid(test_logits)) # logits -> prediction probabilities -> prediction labels
      # 2. Calculate loss and accuracy
      test_loss = loss_fn(test_logits, y_test)
      test_acc = accuracy_fn(y_true=y_test,
                             y_pred=test_pred)

    # Print out what's happening
    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.69480, Accuracy: 39.18% | Test Loss: 0.69152, Test Accuracy: 60.77%
Epoch: 100 | Loss: 0.62913, Accuracy: 62.08% | Test Loss: 0.62482, Test Accuracy: 62.76%
Epoch: 200 | Loss: 0.40490, Accuracy: 85.71% | Test Loss: 0.39801, Test Accuracy: 86.07%
Epoch: 300 | Loss: 0.23929, Accuracy: 92.04% | Test Loss: 0.23891, Test Accuracy: 90.52%
Epoch: 400 | Loss: 0.21588, Accuracy: 92.56% | Test Loss: 0.21687, Test Accuracy: 91.45%
Epoch: 500 | Loss: 0.20547, Accuracy: 93.24% | Test Loss: 0.20726, Test Accuracy: 92.39%
Epoch: 600 | Loss: 0.19755, Accuracy: 93.56% | Test Loss: 0.19972, Test Accuracy: 92.62%
Epoch: 700 | Loss: 0.19111, Accuracy: 93.65% | Test Loss: 0.19348, Test Accuracy: 93.21%
Epoch: 800 | Loss: 0.18538, Accuracy: 93.70% | Test Loss: 0.18775, Test Accuracy: 93.21%
Epoch: 900 | Loss: 0.17953, Accuracy: 93.88% | Test Loss: 0.18189, Test Accuracy: 93.21%
